## Stream Ciphers

## RC4

Implementation details found here https://en.wikipedia.org/wiki/RC4

In [131]:
def KSA(key):
    key_length = len(key)
    S = [0 for i in range(256)]
    for i in range(256):
        S[i] = i
    j = 0
    for i in range(256):
         j = (j + S[i] + key[i % key_length]) % 256
         S[i], S[j] = S[j], S[i]
    return S
         
def PRGA(S):
    i = 0
    j = 0
    while True:
         i = (i + 1) % 256
         j = (j + S[i]) % 256
         S[i], S[j] = S[j], S[i]
         K = S[(S[i] + S[j]) % 256]
         yield K

def RC4(key):
    S = KSA(key.encode())
    return PRGA(S), S

def encrypt(key, plaintext):
    cipher, S = RC4(key)
    plaintext = plaintext.encode()
    ciphertext = []
    for i in range(len(plaintext)):
        ciphertext.append(plaintext[i] ^ next(cipher))
    return plaintext, bytes(ciphertext).hex()

def decrypt(key, ciphertext):
    cipher, S = RC4(key)
    plaintext = []
    for i in range(len(ciphertext)):
        plaintext.append(ciphertext[i] ^ next(cipher))
    return plaintext

    

In [132]:
# Test vectors from Wikipedia
print(encrypt('Key', 'Plaintext'))
print(encrypt('Wiki', 'pedia'))
print(encrypt('Secret', 'Attack at dawn'))

(b'Plaintext', 'bbf316e8d940af0ad3')
(b'pedia', '1021bf0420')
(b'Attack at dawn', '45a01f645fc35b383552544b9bf5')


In [133]:
# Reused Key Attack

key = 'yeet'
plaintext1 = 'I love cryptography'
plaintext2 = 'Attack at dawn!!!!!'

ciphertext1 = encrypt(key, plaintext1)[1]
ciphertext2 = encrypt(key, plaintext2)[1]

print(ciphertext1)
print(ciphertext2)

def xor(a, b):
    return [x ^ y for x, y in zip(a, b)]

reused_key = xor(bytes.fromhex(ciphertext1), bytes.fromhex(ciphertext2))
print(reused_key)

# Suppose we guessed that "cryptography" is contained in one of the plaintext.
guess = b'cryptography'
for i in range(0, len(reused_key)):
    candidate = xor(reused_key[i:i+len(guess)], guess)
    print(bytes(candidate))


aa010ed5a4e3b1ad9ba506a746509805fc2684
a25516dbb1edb1af9dfc12b25e59cb45ad6fdc
[8, 84, 24, 14, 21, 14, 0, 2, 6, 89, 20, 21, 24, 9, 83, 64, 81, 73, 88]
b'k&a~aagpg)|l'
b'7jwezoet8d}a'
b'{|l~tma+uepp'
b'mgwpvi>ftha*'
b'v|yrr6sgyy;9'
b'mr{v-{rjh#(('
b'cp\x7f)`z\x7f{2090'
b'at dawn!!!!!'
b'e+melf42090'
b":flh}<'#(("
b"wgay'/6;9"
b'vjp#4>.*'
b'{{*0%&?'
b'j!9!=7'
b'02(9,'
b'##0('
b'2;!'
b'**'
b';'


In [135]:
# Bit Flipping Attack
# Assumption: Attacker knows part of the plaintext

key = 'yeet'
plaintext = "$1000 transfered"
ciphertext = encrypt(key, plaintext)[1]

print(plaintext)
print(ciphertext)

# At this point the attacker alters ciphertext by XORing with ("$1000 XOR $9000")
a = bytes.fromhex(ciphertext)
payload = xor(b"$1000" , b"$9000")
payload = bytes(xor(a[:len(payload)], bytes(payload))) + a[5:]
d = decrypt(key, payload)
print(bytes(d).decode())

$1000 transfered
c710528ae2a6e5bc88b205b54c458f00
$9000 transfered


# Block Ciphers

## ECB Mode

In [147]:
import base64
from Crypto.Cipher import AES
from Crypto.Util.Padding import pad
from Crypto.Random import get_random_bytes
from Crypto.Util.strxor import strxor


def get_blocks(s, block_size=16):
    return [s[i:i+block_size] for i in range(0, len(s), block_size)]

def aes_ecb_decrypt(ciphertext, key):
    cipher = AES.new(key, AES.MODE_ECB)
    return cipher.decrypt(ciphertext)

def aes_ecb_encrypt(plaintext, key):
    cipher = AES.new(key, AES.MODE_ECB)
    return cipher.encrypt(plaintext)

KEY = get_random_bytes(16)
plaintext = b"I love ECB mode"
ciphertext = aes_ecb_encrypt(pad(plaintext, 16), KEY)

print(plaintext, ciphertext, aes_ecb_decrypt(ciphertext, KEY))

b'I love ECB mode' b'\xaa\x01\xce\x85\x8c\xb0I\xfd\xb1~\xd7\xa5\x86\x92\xbe>' b'I love ECB mode\x01'


## CBC Mode

In [149]:
def aes_cbc_encrypt(plaintext, key, IV):
    blocks = get_blocks(plaintext)
    prev = IV
    ciphertext = b''
    for block in blocks:
        cipher_i = aes_ecb_encrypt(strxor(prev, block), key)
        ciphertext += cipher_i
        prev = cipher_i
    return ciphertext

def aes_cbc_decrypt(ciphertext, key, IV):
    blocks = get_blocks(ciphertext)
    prev = IV
    plaintext = b''
    for block in blocks:
        plain_i = strxor(aes_ecb_decrypt(block, key), prev)
        plaintext += plain_i
        prev = block
    return plaintext

KEY = get_random_bytes(16)
IV = get_random_bytes(16)
plaintext = b"I love CBC mode"
ciphertext = aes_cbc_encrypt(pad(plaintext, 16), KEY, IV)

print(plaintext, ciphertext, aes_cbc_decrypt(ciphertext, KEY, IV))

b'I love CBC mode' b'\xf6\xea\xe9\xc8\xb4\x8f\xd5\x98\x05\xdb\xb9?\xac\xf8 \x94' b'I love CBC mode\x01'


In [157]:
# Assume we have an encryption oracle that appends a secret string after the string.

def encryption_oracle(plaintext):

    secret = b'utflag{this_is_secret_string}'

    plaintext = pad(plaintext + secret, 16)
    return aes_ecb_encrypt(plaintext, KEY)

def get_block_size(upper=100):
    length = len(encryption_oracle(b'A'))
    for i in range(1, upper):
        payload = ('A'*i).encode()
        ciphertext = encryption_oracle(payload)
        if len(ciphertext) != length:
            return len(ciphertext) - length
    return -1

def detect_ecb(block_size):
    payload = ('A'*(2*block_size)).encode()
    ciphertext = encryption_oracle(payload)
    return ciphertext[0:block_size] == ciphertext[block_size:2*block_size]


# Get block size
block_size = get_block_size()
print(block_size)

# Detect ECB
print(detect_ecb(block_size))

# Attack all blocks
secret = ''
prev = ''
for block in range(100):
    for i in range(0, block_size):
        init = 'A' * (block_size - i - 1)
        byte = encryption_oracle(init.encode())[block_size * block:block*block_size + block_size]
        d = {}
        for c in range(256):
            payload =  ('A' * (block_size - i - 1)) + secret + chr(c)
            # print(payload)
            ciphertext = encryption_oracle(payload.encode())
            d[chr(c)] = ciphertext[block_size * block:block*block_size + block_size]
        for key in d.keys():
            if d[key] == byte:
                secret += key
        print(secret)
    if len(prev) == len(secret):
        break

    prev = secret


16
True
u
ut
utf
utfl
utfla
utflag
utflag{
utflag{t
utflag{th
utflag{thi
utflag{this
utflag{this_
utflag{this_i
utflag{this_is
utflag{this_is_
utflag{this_is_s
utflag{this_is_se
utflag{this_is_sec
utflag{this_is_secr
utflag{this_is_secre
utflag{this_is_secret
utflag{this_is_secret_
utflag{this_is_secret_s
utflag{this_is_secret_st
utflag{this_is_secret_str
utflag{this_is_secret_stri
utflag{this_is_secret_strin
utflag{this_is_secret_string
utflag{this_is_secret_string}
utflag{this_is_secret_string}
utflag{this_is_secret_string}
utflag{this_is_secret_string}
utflag{this_is_secret_string}
utflag{this_is_secret_string}
utflag{this_is_secret_string}
utflag{this_is_secret_string}
utflag{this_is_secret_string}
utflag{this_is_secret_string}
utflag{this_is_secret_string}
utflag{this_is_secret_string}
utflag{this_is_secret_string}
utflag{this_is_secret_string}
utflag{this_is_secret_string}
utflag{this_is_secret_string}
utflag{this_is_secret_string}
utflag{this_is_secret_string}
u

## Bit Flipping
https://crypto.stackexchange.com/questions/66085/bit-flipping-attack-on-cbc-mode

## Padding Oracle
https://robertheaton.com/2013/07/29/padding-oracle-attack/